In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.signal import welch
import pywt

# Memuat data yang sudah diproses dari file CSV
processed_df = pd.read_csv('../data/data_1.csv')
print("Data berhasil dimuat dari 'processed_eeg_data.csv'")
print(processed_df.head())

Data berhasil dimuat dari 'processed_eeg_data.csv'
   EXG Channel 0_Window_1  EXG Channel 0_Window_2  EXG Channel 0_Window_3  \
0               -0.056689               -0.009789               -0.028248   
1               -0.042064                0.195527                0.099299   
2               -0.018566                0.294835                0.194787   
3                0.020476                0.252398                0.244480   
4                0.077455                0.122612                0.258575   

   EXG Channel 0_Window_4  EXG Channel 0_Window_5  EXG Channel 0_Window_6  \
0                0.062863               -0.021057                0.130802   
1                0.159499                0.057454                0.315710   
2                0.198253                0.069266                0.424580   
3                0.163947               -0.001853                0.430960   
4                0.093059               -0.107061                0.377047   

   EXG Channel 0_Window

In [17]:
# Ubah DataFrame ke dictionary
filtered_windows_per_channel = {}
for column in processed_df.columns:
    # Pisahkan nama channel dan nomor window dari nama kolom
    channel, window = column.rsplit('_', 1)
    
    # Inisialisasi list untuk channel jika belum ada
    if channel not in filtered_windows_per_channel:
        filtered_windows_per_channel[channel] = []
    
    # Tambahkan window ke channel yang sesuai
    filtered_windows_per_channel[channel].append(processed_df[column].values)

# Konversi list of arrays ke array of arrays
for channel in filtered_windows_per_channel:
    filtered_windows_per_channel[channel] = np.array(filtered_windows_per_channel[channel])

# Contoh: Tampilkan data untuk EXG Channel 0
# print("Data untuk EXG Channel 0:", filtered_windows_per_channel['EXG Channel 0_Window_10'])

In [18]:
fs = 250
def extract_time_domain_features(window):
    features = {
        'mean': np.mean(window),
        'variance': np.var(window),
        'std_dev': np.std(window),
        'rms': np.sqrt(np.mean(np.square(window))),
        'zero_crossing_rate': np.sum(np.abs(np.diff(np.sign(window)))) / len(window)
    }
    return features

In [19]:
def extract_frequency_domain_features(window, fs):
    # Hitung FFT
    n = len(window)
    yf = fft(window)
    xf = np.linspace(0.0, 1.0 / (2.0 * (1.0 / fs)), n // 2)
    
    # Hitung Power Spectral Density (PSD)
    freqs, psd = welch(window, fs, nperseg=len(window))
    
    # Hitung frekuensi dominan
    dominant_freq = freqs[np.argmax(psd)]
    
    # Hitung band power
    def band_power(freqs, psd, low, high):
        idx = np.logical_and(freqs >= low, freqs <= high)
        return np.trapz(psd[idx], freqs[idx])
    
    delta_power = band_power(freqs, psd, 0.5, 4)
    theta_power = band_power(freqs, psd, 4, 8)
    alpha_power = band_power(freqs, psd, 8, 13)
    beta_power = band_power(freqs, psd, 13, 30)
    gamma_power = band_power(freqs, psd, 30, 100)
    
    features = {
        'dominant_freq': dominant_freq,
        'delta_power': delta_power,
        'theta_power': theta_power,
        'alpha_power': alpha_power,
        'beta_power': beta_power,
        'gamma_power': gamma_power
    }
    return features

In [20]:
def extract_time_frequency_domain_features(window, wavelet='db4', level=3):
    # Lakukan wavelet transform
    coeffs = pywt.wavedec(window, wavelet, level=level)
    
    # Ekstrak koefisien approximation dan detail
    features = {
        'approximation': coeffs[0],
        'detail_1': coeffs[1],
        'detail_2': coeffs[2],
        'detail_3': coeffs[3]
    }
    return features

In [26]:
time_domain_features = {}
frequency_domain_features = {}
time_frequency_domain_features = {}

for channel, windows in filtered_windows_per_channel.items():
    time_domain_features[channel] = []
    frequency_domain_features[channel] = []
    time_frequency_domain_features[channel] = []
    
    for window in windows:
        # Ekstrak fitur domain waktu
        time_feat = extract_time_domain_features(window)
        time_domain_features[channel].append(time_feat)
        
        # Ekstrak fitur domain frekuensi
        freq_feat = extract_frequency_domain_features(window, fs)
        frequency_domain_features[channel].append(freq_feat)
        
        # Ekstrak fitur domain waktu-frekuensi
        tf_feat = extract_time_frequency_domain_features(window)
        time_frequency_domain_features[channel].append(tf_feat)

# Contoh: Tampilkan fitur untuk window pertama dari EXG Channel 0
print("Time Domain Features (EXG Channel 0:", time_domain_features['EXG Channel 0_Window'])
print("Frequency Domain Features (EXG Channel 0:", frequency_domain_features['EXG Channel 0_Window'])
print("Time-Frequency Domain Features (EXG Channel 0:", time_frequency_domain_features['EXG Channel 0_Window'])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_160468\2518406232.py:16: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(psd[idx], freqs[idx])


Time Domain Features (EXG Channel 0: [{'mean': np.float64(0.06645562663565444), 'variance': np.float64(0.022591255407266687), 'std_dev': np.float64(0.15030387688701408), 'rms': np.float64(0.16433990908724572), 'zero_crossing_rate': np.float64(0.224)}, {'mean': np.float64(0.028033590399872934), 'variance': np.float64(0.020671448034052898), 'std_dev': np.float64(0.14377568651915001), 'rms': np.float64(0.14648320799586806), 'zero_crossing_rate': np.float64(0.264)}, {'mean': np.float64(0.002952536513883281), 'variance': np.float64(0.04423055585182536), 'std_dev': np.float64(0.21031061754420616), 'rms': np.float64(0.21033134175317564), 'zero_crossing_rate': np.float64(0.136)}, {'mean': np.float64(-0.01139361185092259), 'variance': np.float64(0.017059965528565996), 'std_dev': np.float64(0.13061380297872807), 'rms': np.float64(0.13110980100501823), 'zero_crossing_rate': np.float64(0.208)}, {'mean': np.float64(0.03311344418726473), 'variance': np.float64(0.02405065513335446), 'std_dev': np.flo

In [29]:
# Buat DataFrame untuk fitur domain waktu
time_domain_features_list = []

for channel in time_domain_features.keys():
    for i, features in enumerate(time_domain_features[channel]):
        features['channel'] = channel
        features['window'] = i + 1
        time_domain_features_list.append(features)

# Gabungkan semua fitur ke dalam satu DataFrame
time_domain_features_df = pd.DataFrame(time_domain_features_list)

# Simpan ke file CSV
time_domain_features_df.to_csv('time_domain_features.csv', index=False)
print("Fitur domain waktu berhasil disimpan ke 'time_domain_features.csv'")

Fitur domain waktu berhasil disimpan ke 'time_domain_features.csv'


In [30]:
# Buat DataFrame untuk fitur domain frekuensi
frequency_domain_features_list = []

for channel in frequency_domain_features.keys():
    for i, features in enumerate(frequency_domain_features[channel]):
        features['channel'] = channel
        features['window'] = i + 1
        frequency_domain_features_list.append(features)

# Gabungkan semua fitur ke dalam satu DataFrame
frequency_domain_features_df = pd.DataFrame(frequency_domain_features_list)

# Simpan ke file CSV
frequency_domain_features_df.to_csv('frequency_domain_features.csv', index=False)
print("Fitur domain frekuensi berhasil disimpan ke 'frequency_domain_features.csv'")

Fitur domain frekuensi berhasil disimpan ke 'frequency_domain_features.csv'


In [31]:
# Buat DataFrame untuk fitur domain waktu-frekuensi
time_frequency_domain_features_list = []

for channel in time_frequency_domain_features.keys():
    for i, features in enumerate(time_frequency_domain_features[channel]):
        features['channel'] = channel
        features['window'] = i + 1
        time_frequency_domain_features_list.append(features)

# Gabungkan semua fitur ke dalam satu DataFrame
time_frequency_domain_features_df = pd.DataFrame(time_frequency_domain_features_list)

# Simpan ke file CSV
time_frequency_domain_features_df.to_csv('time_frequency_domain_features.csv', index=False)
print("Fitur domain waktu-frekuensi berhasil disimpan ke 'time_frequency_domain_features.csv'")

Fitur domain waktu-frekuensi berhasil disimpan ke 'time_frequency_domain_features.csv'
